In [92]:
import numpy as np
import scipy
from scipy.integrate import quad
from scipy.stats import multivariate_normal, norm
from scipy.optimize import minimize

# Equivalent to R's logit function
def logit(x):
    return np.log(x / (1 - x))

# Equivalent to R's expit function
def expit(x):
    return 1 / (1 + np.exp(-x))
# Bivariate normal PDF
# Equivalent to R's biv_pdf function
def biv_pdf(x, y, mu, sigma, rho, log=False):
    rho_part = 1 - rho**2
    if rho_part <= 0:
        rho_part = 0.000001  # To prevent division by zero or negative values

    factor_part = 2 * np.pi * sigma[0] * sigma[1] * np.sqrt(rho_part)
    exp_part = ((x - mu[0]) / sigma[0])**2 + ((y - mu[1]) / sigma[1])**2
    exp_part -= 2 * rho * (x - mu[0]) * (y - mu[1]) / (sigma[0] * sigma[1])
    exp_part = -exp_part / (2 * rho_part)

    if log:
        return exp_part - np.log(factor_part)
    else:
        return np.exp(exp_part) / factor_part

# Equivalent to R's dmg_int_inner function
def dmg_int_inner(x, y, mu, sigma, rho, eta, alpha, l):
    sd_x = sigma[0]
    sd_y = sigma[1]
    # Adjusted value of y
    adjusted_y = y + (mu[1] / mu[0]) * alpha * (l / eta - x)
    return biv_pdf(x, adjusted_y, mu, sigma, rho)

# Equivalent to R's dmg_int function
def dmg_int(y, mu, sigma, rho, eta, alpha, l):
    # Use scipy's quad to perform the integration
    result, _ = quad(dmg_int_inner, l, l / eta, args=(y, mu, sigma, rho, eta, alpha, l), epsabs=1e-5)
    return result


def dmg_model(samples, eta, alpha, l, mu):
    # Check if l > samples[:, 0] * eta for each row
    condition = l > samples[:, 0] * eta
    adjusted_values = samples[:, 1] - (mu[1] / mu[0]) * alpha * (l / eta - samples[:, 0])
    
    # If condition is True, calculate damage; otherwise, use undamaged value
    return np.where(condition, adjusted_values, samples[:, 1])
def int_function(y_star, mu, sigma, rho, l):
    a_l = (l - mu[0] - rho * (sigma[0] / sigma[1]) * (y_star - mu[1])) / (sigma[0] * np.sqrt(1 - rho**2))
    return norm.pdf(y_star, loc=mu[1], scale=sigma[1]) * (1 - norm.cdf(a_l))


def PFY_lik(mu, sigma, rho, eta, alpha, l, data):
    part_1 = np.sum(norm.logpdf(data[data[:, 2] == 1, 0], loc=mu[0], scale=sigma[0]))
    
    dmg_int_values = np.array([dmg_int(y, mu, sigma, rho, eta, alpha, l) for y in data[data[:, 2] == 0, 1]])
    int_function_values = np.array([int_function(y, mu, sigma, rho, 1/eta * l) for y in data[data[:, 2] == 0, 1]])
    
    part_2 = np.sum(np.log(dmg_int_values + int_function_values))
    
    return part_1 + part_2


def pl_gen(mu, sigma, rho, eta, alpha, l, N):
    # Covariance matrix
    cov_matrix = np.array([
        [sigma[0]**2, sigma[0] * sigma[1] * rho],
        [sigma[0] * sigma[1] * rho, sigma[1]**2]
    ])
    # Generate samples from bivariate normal distribution
    samples = np.random.multivariate_normal(mean=mu, cov=cov_matrix, size=N)
    res = np.zeros((samples.shape[0], samples.shape[1] + 1))
    
    # Fail in the proof loading
    fail_ids = samples[:, 0] < l
    res[fail_ids, 0] = samples[fail_ids, 0]
    res[fail_ids, 2] = 1
    
    # Survive in the proof loading
    survive_ids = samples[:, 0] >= l
    res[survive_ids, 1] = dmg_model(samples[survive_ids], eta, alpha, l, mu)
    res[survive_ids, 2] = 0
    
    return res


In [121]:
# Example usage (requires biv_pdf to be implemented)
mu = [45, 5.5]
sigma = [13, 1]
rho = .7
eta = .7
alpha = 0

# rho = correlation_coefficient
N = 87

R_pf = norm.ppf([0.2, 0.4, 0.6], loc=mu[0], scale=sigma[0])
T_pf = norm.ppf([0.2, 0.4, 0.6], loc=mu[1], scale=sigma[1])
l = R_pf[1]
pf_data = pl_gen(mu, sigma, rho, eta, alpha, l, N)
#print(pf_data)

In [122]:
theta0  = [mu[0],mu[1],
           sigma[0],sigma[1],logit(rho),logit(eta),alpha]


In [123]:
PFY_lik(mu, sigma, rho, eta, alpha, l, pf_data)

-242.5807017830876

In [124]:
def alpha_checkR(theta, R_group, T_group, R_pl, T_pl, shoulder_group):
    mu = theta[0:2]
    sigma = theta[2:4]
    rho = expit(theta[4])
    eta = expit(theta[5])
    alpha = theta[6]
    
    # Likelihood calculations
    lik = (PFY_lik(mu, sigma, rho, 1, 0, R_pl, R_group) +
           PFY_lik([mu[1], mu[0]], [sigma[1], sigma[0]], rho, eta, alpha, T_pl, T_group) +
           np.sum(norm.logpdf(shoulder_group[0], loc=mu[0], scale=sigma[0])) +
           np.sum(norm.logpdf(shoulder_group[1], loc=mu[1], scale=sigma[1])))
    
    # Handle infinite likelihood
    if np.isinf(lik):
        lik = -10000
    
    return -1 * lik

def single_fitalpha(theta, group, group_pl, group_name, shoulder_group):
    mu = theta[0:2]
    sigma = theta[2:4]
    rho = theta[4]
    eta = theta[5]
    alpha = theta[6]
    
    if group_name == "R":
        lik = PFY_lik(mu, sigma, rho, eta, alpha, group_pl, group)
    elif group_name == "T":
        lik = PFY_lik([mu[1], mu[0]], [sigma[1], sigma[0]], rho, eta, alpha, group_pl, group)
    
    lik += (np.sum(norm.logpdf(shoulder_group[0], loc=mu[0], scale=sigma[0])) +
            np.sum(norm.logpdf(shoulder_group[1], loc=mu[1], scale=sigma[1])))
    
    if np.isinf(lik):
        lik = -10000
    
    return -1 * lik

# Equivalent to R's single_alpha0 function
def single_alpha0(theta, group, group_pl, group_name, shoulder_group):
    mu = theta[0:2]
    sigma = theta[2:4]
    rho = expit(theta[4])
    
    if group_name == "R":
        lik = PFY_lik(mu, sigma, rho, 1, 0, group_pl, group)
    elif group_name == "T":
        lik = PFY_lik([mu[1], mu[0]], [sigma[1], sigma[0]], rho, 1, 0, group_pl, group)
    
    lik += (np.sum(norm.logpdf(shoulder_group[0], loc=mu[0], scale=sigma[0])) +
            np.sum(norm.logpdf(shoulder_group[1], loc=mu[1], scale=sigma[1])))
    
    if np.isinf(lik):
        lik = -10000
    
    return -1 * lik


def llr_sim(data, shoulder_group, theta0, group_pl):
    # Optimization step for single_alpha0
    optim_checkR = minimize(single_alpha0, theta0[0:5], args=(data, group_pl, "R", shoulder_group))
    llr_checkR = 2 * optim_checkR.fun
    
    # Optimization step for single_fitalpha
    bounds = [(30, 70), (0.1, 40), (0.1, 30), (0.1, 10), (0.03, 0.97), (0.03, 0.97), (-2, 10)]
    optimout = minimize(single_fitalpha, theta0, 
                        args=(data, group_pl, "R", shoulder_group), 
                        method="L-BFGS-B", bounds=bounds)
    llr_full = 2 * optimout.fun
    
    # Calculate LLR statistic
    llr_stat = llr_checkR - llr_full
    return llr_stat

def ecdf(x):
    x = np.sort(x)
    n = len(x)
    def _ecdf(v):
        # side='right' because we want Pr(x <= v)
        return (np.searchsorted(x, v, side='right') + 1) / n
    return _ecdf

In [125]:
np.random.seed(42)  # Set seed for reproducibility
pf_data = pl_gen(mu, sigma, rho, eta, alpha, l, N)
R100_data = np.random.normal(loc=mu[0], scale=sigma[0], size=2 * N)
T100_data = np.random.normal(loc=mu[1], scale=sigma[1], size=2 * N)
shoulder_group = [R100_data, T100_data]


In [158]:
def lrt_fit(jj,theta0):
    np.random.seed(jj)  # Set seed for reproducibility
    pf_data = pl_gen(mu, sigma, rho, eta, alpha, l, N)
    R100_data = np.random.normal(loc=mu[0], scale=sigma[0], size=2 * N)
    T100_data = np.random.normal(loc=mu[1], scale=sigma[1], size=2 * N)
    shoulder_group = [R100_data, T100_data]



    optim_checkR = minimize(single_alpha0, theta0[0:5], args=(pf_data, l, "R", shoulder_group),
                           method = "Nelder-Mead")
    llr_checkR = 2 * optim_checkR.fun
    theta_est = optim_checkR.x
    # # Optimization step for single_fitalpha
    bounds = [(30, 70), (0.1, 40), (0.1, 30), (0.1, 10), (0.01, 0.99), (0.01, 0.99), (-2, 10)]
    optimout = minimize(single_fitalpha, theta0, 
                        args=(pf_data,l, "R", shoulder_group), 
                        method="L-BFGS-B", bounds=bounds)
    llr_full = 2 * optimout.fun
    end_time = time.time()
    llr_stat = llr_checkR - llr_full
    return [jj,llr_stat,*theta_est.tolist()]


In [159]:
import joblib
N_rep = 200
numbers = range(N_rep)
results = joblib.Parallel(n_jobs=8)(joblib.delayed(lrt_fit)(num,theta0) for num in numbers)

/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7257/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7257/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7257/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7257/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7257/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7257/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7257/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7257/1550408679.p

/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7257/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7257/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7257/1550408679.py:64: RuntimeWarning: divide by zero encountered in log


In [162]:
res = np.array(results)
res[:,2]

array([43.54396558, 45.41377216, 44.3168373 , 46.68082367, 46.44134282,
       44.45749392, 46.04969592, 44.43757652, 45.05130379, 45.0889602 ,
       44.94683952, 43.94345879, 44.909987  , 45.42505887, 45.20033938,
       45.40941066, 44.0394525 , 43.69968144, 43.4313857 , 45.50214313,
       44.93280527, 45.25230584, 45.39079399, 44.95205385, 43.94833298,
       45.56140189, 46.00168123, 44.18681632, 44.56307872, 44.49025411,
       44.63280801, 46.44548567, 44.58159266, 46.23224831, 44.69303591,
       43.88307181, 44.47211964, 43.89304276, 44.64640596, 45.0627134 ,
       45.08280457, 44.27081703, 46.0826102 , 44.85537605, 45.3386103 ,
       46.43506338, 45.50235179, 44.29222324, 45.6616406 , 45.41339637,
       45.07989374, 43.35055011, 45.43480995, 43.55038197, 45.69593676,
       44.42460709, 44.93714182, 44.22610017, 43.04713178, 44.51500205,
       44.11147025, 44.76727139, 45.10980429, 45.54240095, 46.14263053,
       45.52887262, 45.24238655, 45.69789301, 46.34073215, 43.27

In [153]:
res[41,]

array([   41.        ,            nan,  -557.42515301,  -148.02179079,
        -437.16340988, -1247.56541804,  -161.95517564])

In [154]:
np.random.seed(4)  # Set seed for reproducibility
pf_data = pl_gen(mu, sigma, rho, eta, alpha, l, N)
R100_data = np.random.normal(loc=mu[0], scale=sigma[0], size=2 * N)
T100_data = np.random.normal(loc=mu[1], scale=sigma[1], size=2 * N)
shoulder_group = [R100_data, T100_data]



optim_checkR = minimize(single_alpha0, theta0[0:5], args=(pf_data, l, "R", shoulder_group))
llr_checkR = 2 * optim_checkR.fun
theta_est = optim_checkR.x
# # Optimization step for single_fitalpha
bounds = [(30, 70), (0.1, 40), (0.1, 30), (0.1, 10), (0.01, 0.99), (0.01, 0.99), (-2, 10)]
optimout = minimize(single_fitalpha, theta0, 
                    args=(pf_data,l, "R", shoulder_group), 
                    method="L-BFGS-B", bounds=bounds)
llr_full = 2 * optimout.fun
end_time = time.time()
llr_stat = llr_checkR - llr_full

/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7257/1550408679.py:13: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))
/var/folders/0x/z947m7157nn9jf8zfnx8w9zw0000gn/T/ipykernel_7257/1550408679.py:64: RuntimeWarning: divide by zero encountered in log
  part_2 = np.sum(np.log(dmg_int_values + int_function_values))


In [157]:
optim_checkR = minimize(single_alpha0, theta0[0:5], args=(pf_data, l, "R", shoulder_group),
                       method = "Nelder-Mead")
optim_checkR 

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 1169.273354416332
             x: [ 4.644e+01  5.533e+00  1.384e+01  9.039e-01  2.282e-01]
           nit: 208
          nfev: 343
 final_simplex: (array([[ 4.644e+01,  5.533e+00, ...,  9.039e-01,
                         2.282e-01],
                       [ 4.644e+01,  5.533e+00, ...,  9.039e-01,
                         2.281e-01],
                       ...,
                       [ 4.644e+01,  5.533e+00, ...,  9.039e-01,
                         2.281e-01],
                       [ 4.644e+01,  5.533e+00, ...,  9.039e-01,
                         2.281e-01]]), array([ 1.169e+03,  1.169e+03,  1.169e+03,  1.169e+03,
                        1.169e+03,  1.169e+03]))

In [111]:
# def ecdf(data):
#     # Rank data and divide by the number of points to calculate ECDF
#     sorted_data = np.sort(data)
#     return lambda x: np.searchsorted(sorted_data, x, side='right') / len(data)



np.random.seed(1)
data = np.random.normal(loc=0, scale=1, size=2000)
ecdf(data)(np.nan)

1.0

In [74]:
res = np.zeros((200,7))

jj = 0
np.random.seed(jj)  # Set seed for reproducibility
pf_data = pl_gen(mu, sigma, rho, eta, alpha, l, N)
R100_data = np.random.normal(loc=mu[0], scale=sigma[0], size=2 * N)
T100_data = np.random.normal(loc=mu[1], scale=sigma[1], size=2 * N)
shoulder_group = [R100_data, T100_data]



optim_checkR = minimize(single_alpha0, theta0[0:5], args=(pf_data, l, "R", shoulder_group))
llr_checkR = 2 * optim_checkR.fun
theta_est = optim_checkR.x
# # Optimization step for single_fitalpha
bounds = [(30, 70), (0.1, 40), (0.1, 30), (0.1, 10), (0.01, 0.99), (0.01, 0.99), (-2, 10)]
optimout = minimize(single_fitalpha, theta0, 
                    args=(pf_data,l, "R", shoulder_group), 
                    method="L-BFGS-B", bounds=bounds)
llr_full = 2 * optimout.fun
end_time = time.time()
llr_stat = llr_checkR - llr_full
llr_stat 
res[jj,:] = [jj,llr_stat,*theta_est.tolist()]


array([ 0.        ,  6.13204411, 43.63029646,  5.43060899, 12.71142531,
        1.02380281,  1.30212692])